In [13]:
import pandas as pd
import numpy as np
import math
length = 510

In [2]:
annot_df = pd.read_csv('../data/annotation/cannonical_annotation.csv',sep = '\t',header = 0,index_col = 0)
transcript_df = pd.read_csv('../data/annotation/annot_label.csv',sep = '\t',header = 0)
transcript_df['length'] = np.absolute(transcript_df['Start']-transcript_df['End'])
#How many sequences if we fully chop transcriptome
np.sum(np.ceil(transcript_df['length']/length))
annot_df = annot_df[annot_df['Annotation'].isin(['exon','UTR',])]
annot_df['GeneID']=pd.Categorical(annot_df['GeneID'],transcript_df['Gene'])
annot_df = annot_df.sort_values(by=['Chrom','GeneID','Start']).reset_index()

In [3]:
reverse_strand = {'+':'-','-':'+'}

chrom = []
strand = []
start_l = []
end_l = []
gene = []
for index,row in transcript_df.iterrows():
    start = row['Start']
    end = row['End']
    split = [(round(length*i)+start, round(length*(i+1))+start) for i in range(int(math.ceil((end-start)/length)))]
    for entry in split:
        chrom.append(row['Chr'])
        strand.append(reverse_strand[row['Strand']])
        start_l.append(entry[0])
        end_l.append(entry[1])
        gene.append(row['Gene'])
    

In [4]:
bed_df = pd.DataFrame({'Chr':chrom,'Start':start_l,'End':end_l,'Strand':strand,'Gene':gene})
bed_df['Gene']=pd.Categorical(bed_df['Gene'],transcript_df['Gene'])
bed_df = bed_df.sort_values(by=['Chr','Gene','Start'])

In [5]:
bed_pointer = 0
annot_pointer = 0
label = np.empty(len(bed_df), dtype=object)
label[...] = [set()for _ in range(label.shape[0])]
#while bed_df.iloc[bed_pointer]['Gene'] == annot_df.iloc[annot_pointer]['GeneID']:
while bed_pointer < len(bed_df)-1  or annot_pointer < len(annot_df)-1:
    bed_row = bed_df.iloc[bed_pointer]
    annot_row = annot_df.iloc[annot_pointer]
    #chromosome check:
    if bed_row['Chr'] < annot_row['Chrom']:
        bed_pointer += 1
        continue
    elif bed_row['Chr'] > annot_row['Chrom']:
        annot_pointer += 1
        continue
    #not overlapping， move pointer 
    if bed_row['End'] <= annot_row['Start']:
        bed_pointer += 1
        continue
    elif bed_row['Start'] >= annot_row['End']:
        annot_pointer += 1
        continue
    #overlapping
    if max(bed_row['Start'], annot_row['Start']) <= min(bed_row['End'], annot_row['End']):
        #complete overlap:
        if (bed_row['Start'] >= annot_row['Start']) and (bed_row['End'] <= annot_row['End']):
            label[bed_pointer].add(annot_row['Annotation'])
            bed_pointer += 1 
        #partial overlap
        elif bed_row['Start'] <= annot_row['Start']:
            label[bed_pointer].update(['junction',annot_row['Annotation']])
            bed_pointer += 1
        #partial overlap
        elif bed_row['End'] >= annot_row['End']:
            label[bed_pointer].update([annot_row['Annotation'],'junction'])
            annot_pointer += 1
               

In [6]:
label_list = [list(ele) for ele in label]
label = [','.join(ele) for ele in label_list]
bed_df['Label'] = label
bed_df['Label'] = bed_df['Label'].replace(r'^\s*$', 'intron', regex=True)

In [7]:
bed_df.to_csv('../data/pre-train/'+str(length)+'/rna_selection.bed',sep = '\t', index=False, header = False)

In [8]:
from tqdm import tqdm

fasta = open('../data/pre-train/'+str(length)+'/rna_seq.fa', 'r')
lines = fasta.readlines()
seq = []
# Strips the newline character
for line in tqdm(lines):
    if line[0] == '>':
        next
    else: 
        seq.append(line.strip().upper())

100%|██████████| 7669382/7669382 [00:05<00:00, 1511546.34it/s]


In [9]:
import random
random.shuffle(seq)
data_length = len(seq)
train_data = seq[:int(data_length*0.8)]
valid_data = seq[int(data_length*0.8):int(data_length*0.9)]
test_data = seq[int(data_length*0.9):]

In [14]:
import h5py
import numpy as np
file_name = '../data/pre-train/'+str(length)+'/rna_seq.h5'
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('train',data = train_data)
h5f.create_dataset('valid',data = valid_data)
h5f.create_dataset('test',data = test_data)
h5f.close()

In [7]:
import h5py
import numpy as np
file_name = '../data/pre-train/rna_seq.h5'
h5f = h5py.File(file_name, 'r')['seq']

In [8]:
import re
pattern = '[^ACGTN]'
count = 0
for seq in h5f:
    seq = seq.decode('utf-8')
    search = re.search(pattern, seq, flags=0)
    if  search != None:
        count+=1
        print(search)
        break

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
bed_df = pd.read_csv('../data/pre-train/region_selection.bed',sep='\t',index_col=None)

bed_df['Label'].unique()

In [ ]:
bed_df[bed_df['Label'].str.contains("intron")]

Count of reads containing each label

'UTR':19953

'exon':363682

'junction': 284194

'intron': 3455744



Not balanced. Sub-sample?

Total read :3819756

In [ ]:
len('CTTCTATTTATTTATTTATTTATTTATTTGTTTGTTTTAGAAGATTCTATGTTAATATTTTATGTGT')